# Task 3: Comprehensive Hypothesis Testing Notebook

## Overview
This notebook analyzes historical insurance data (Feb 2014 - Aug 2015) for AlphaCare Insurance Solutions (ACIS) to test 28 null hypotheses on risk drivers. Risk metrics include Claim Frequency, Claim Severity, and Margin. Analysis and visualizations are performed here using imported functions.

## Objectives
- Load and preprocess data.
- Calculate overall metrics.
- Test hypotheses across 28 variables.
- Generate and interpret visualizations.
- Provide business recommendations.

## Prerequisites
- Install dependencies from requirements.txt.
- Ensure data/raw/insurance_data.txt exists.

In [1]:
# Cell 1: Import Libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats  # Added for statistical tests
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # Added for Tukey HSD test

# Add src to Python path (project root aware)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print("src_path added to sys.path:", src_path)
print("stats exists:", os.path.exists(os.path.join(src_path, "stats")))
print("hypothesis_testing.py exists:", os.path.exists(os.path.join(src_path, "stats", "hypothesis_testing.py")))

from stats.hypothesis_testing import calculate_metrics, segment_data, test_hypothesis, visualize_results

# Set plotting style for better visuals
sns.set_style("whitegrid")
sns.set_palette("husl")

# Documentation
# - sys/os: Used to add src to Python path dynamically, resolving ModuleNotFoundError in VS Code.
# - pandas: For data manipulation and analysis.
# - matplotlib/seaborn: For data visualization.
# - scipy.stats: Provides statistical functions (e.g., t-test, ANOVA) used in hypothesis testing.
# - statsmodels.stats.multicomp: Provides pairwise_tukeyhsd for post-hoc analysis.
# - stats.hypothesis_testing: Custom module with testing functions, imported relative to src.
# - plt.style.use/seaborn.set_palette: Enhances plot aesthetics.

src_path added to sys.path: c:\Users\hp\Desktop\projects\10 Acadamy -KAIM5\Insurance-Risk-Analytics-Predictive-Modeling\src
stats exists: True
hypothesis_testing.py exists: True


In [2]:
# Cell 2: Load and Preprocess Data
# Load the insurance dataset with pipe delimiter and parse dates
data_path = os.path.join(project_root, "data", "raw", "insurance_data.txt")
df = pd.read_csv(data_path, sep="|", parse_dates=["TransactionMonth"])

# Drop rows with missing values for key columns (only those that exist)
columns_to_check = ["TotalClaims", "TotalPremium", "Province", "PostalCode", "Gender", "UnderwrittenCoverID", "PolicyID",
                    "IsVATRegistered", "Citizenship", "LegalType", "MaritalStatus", "Language", "Bank", "Country",
                    "MainCrestaZone", "SubCrestaZone", "ItemType", "VehicleType", "AlarmImmobiliser", "make",
                    "bodytype", "RegistrationYear", "TermFrequency", "CoverCategory", "Product", "StatutoryClass",
                    "SumInsured"]
existing_columns = [col for col in columns_to_check if col in df.columns]
missing_columns = [col for col in columns_to_check if col not in df.columns]
if missing_columns:
    print("Warning: The following columns are missing from the data and will be ignored:", missing_columns)
df = df.dropna(subset=existing_columns)

# Bin continuous variables for categorical testing
df["TotalPremiumBin"] = pd.qcut(df["TotalPremium"], 4, labels=["Low", "Medium-Low", "Medium-High", "High"])
df["SumInsuredBin"] = pd.qcut(df["SumInsured"], 4, labels=["Low", "Medium-Low", "Medium-High", "High"])
df["RegistrationYearBin"] = pd.cut(df["RegistrationYear"], bins=5, labels=["Very Old", "Old", "Average", "New", "Very New"])

# Display basic info to verify data loading
print("Data Shape:", df.shape)
print("Columns:", df.columns.tolist())

C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2982703745.py:4: DtypeWarning: Columns (0,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, sep="|", parse_dates=["TransactionMonth"])


Data Shape: (845326, 55)
Columns: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims', 'TotalPremiumBin', 'SumInsuredBin', 'RegistrationYearBin']


In [3]:
# Cell 3: Calculate Overall Metrics
# Compute baseline metrics for the entire dataset
metrics = calculate_metrics(df)

# Display results
print("Overall Metrics:")
for metric, value in metrics.items():
    print("{0} {1:.4f}".format(metric.replace("_", " ").title(), value))

# Documentation
# - calculate_metrics: Computes Claim Frequency, Severity, and Margin.
# - print: Formats and displays metrics.

Overall Metrics:
Claim Frequency 0.0030
Claim Severity 22895.4291
Margin -7455504.7379


In [5]:
# Cell 4: Define Hypotheses with A/B Splits
# Define hypotheses with specific A/B groups for segmentation
hypotheses = []
base_hypotheses = [
    (["Province"], ["Gauteng", "Western Cape"], "TotalClaims", False, "Claim Severity by Province", "box"),  # Top 2 provinces by population
    (["PostalCode"], df["PostalCode"].value_counts().head(2).index.tolist(), "TotalClaims", False, "Claim Severity by Zip Code", "box"),  # Dynamic top 2 postal codes
    (["Gender"], ["Male", "Female"], "TotalClaims", True, "Claim Frequency by Gender", "box"),  # Binary feature
    (["PostalCode"], df["PostalCode"].value_counts().head(2).index.tolist(), pd.Series(df["TotalPremium"] - df["TotalClaims"]), False, "Margin by Zip Code", "box"),  # Dynamic top 2 postal codes
    (["UnderwrittenCoverID"], [], "TotalClaims", True, "Claim Frequency by Cover ID", "box"),  # Dynamic top 2 cover IDs
    (["PolicyID"], [], "TotalClaims", False, "Claim Severity by Policy ID", "box"),  # Dynamic top 2 policy IDs
    (["IsVATRegistered"], [0, 1], "TotalClaims", True, "Claim Frequency by VAT Status", "box"),  # Binary feature
    (["Citizenship"], [], "TotalClaims", True, "Claim Severity by Citizenship", "box"),  # Dynamic top 2 citizenships
    (["LegalType"], [], "TotalClaims", True, "Claim Frequency by Legal Type", "box"),  # Dynamic top 2 legal types
    (["MaritalStatus"], [], pd.Series(df["TotalPremium"] - df["TotalClaims"]), False, "Margin by Marital Status", "box"),  # Dynamic top 2 marital statuses
    (["Language"], [], "TotalClaims", True, "Claim Frequency by Language", "box"),  # Dynamic top 2 languages
    (["Bank"], [], "TotalClaims", False, "Claim Severity by Bank", "box"),  # Dynamic top 2 banks
    (["Country"], [], "TotalClaims", True, "Claim Frequency by Country", "box"),  # Dynamic top 2 countries
    (["MainCrestaZone"], [], "TotalClaims", False, "Claim Severity by Main Cresta Zone", "box"),  # Dynamic top 2 main zones
    (["SubCrestaZone"], [], "TotalClaims", True, "Claim Frequency by Sub Cresta Zone", "box"),  # Dynamic top 2 sub-zones
    (["ItemType"], [], "TotalClaims", True, "Claim Frequency by Item Type", "box"),  # Dynamic top 2 item types
    (["VehicleType"], [], "TotalClaims", False, "Claim Severity by Vehicle Type", "box"),  # Dynamic top 2 vehicle types
    (["AlarmImmobiliser"], [0, 1], "TotalClaims", True, "Claim Frequency by Alarm", "box"),  # Binary feature
    (["make"], [], "TotalClaims", False, "Claim Severity by Make", "box"),  # Dynamic top 2 makes
    (["bodytype"], [], "TotalClaims", True, "Claim Frequency by Body Type", "box"),  # Dynamic top 2 body types
    (["RegistrationYear"], [], pd.Series(df["TotalPremium"] - df["TotalClaims"]), False, "Margin by Registration Year", "box"),  # Dynamic top 2 years
    (["TermFrequency"], [0, 1], "TotalClaims", True, "Claim Frequency by Term Frequency", "box"),  # Binary feature (corrected to explicit 0, 1)
    (["CoverCategory"], [], "TotalClaims", False, "Claim Severity by Cover Category", "box"),  # Dynamic top 2 categories
    (["Product"], [], "TotalClaims", True, "Claim Frequency by Product", "box"),  # Dynamic top 2 products
    (["StatutoryClass"], [], pd.Series(df["TotalPremium"] - df["TotalClaims"]), False, "Margin by Statutory Class", "box"),  # Dynamic top 2 classes
    (["TotalPremiumBin"], [], "TotalClaims", True, "Claim Frequency by Premium Level", "box"),  # Dynamic top 2 bins
    (["SumInsuredBin"], [], "TotalClaims", False, "Claim Severity by Sum Insured", "box"),  # Dynamic top 2 bins
    (["TransactionMonth"], [], "TotalClaims", False, "Claim Severity by Month", "heatmap"),  # Dynamic top 2 months
]

for group_col, groups, metric_col, is_cat, title, plot_type in base_hypotheses:
    try:
        if len(groups) == 2 and isinstance(groups[0], str):  # Predefined A/B pairs
            hypotheses.append((group_col, groups, metric_col, is_cat, title, plot_type))
        else:  # Dynamic A/B pairs
            top_two = df[group_col[0]].value_counts().head(2).index.tolist()
            if len(top_two) == 2:
                hypotheses.append((group_col, top_two, metric_col, is_cat, title, plot_type))
            else:
                print(f"Warning: Insufficient unique values for {group_col[0]} in {title}, skipping.")
    except KeyError as e:
        print(f"Warning: Column {group_col[0]} ({str(e)}) not found in df, skipping {title}.")

# Documentation
# - hypotheses: List of tuples defining A/B groups for each test, with dynamic selection of top 2 categories for multi-class features.
# - group_col: List of column name(s) or specific groups [A, B] for segmentation.
# - metric_col: Column or Series for the metric (TotalClaims or Margin).
# - is_cat: Boolean for categorical test (chi-squared) vs. numerical (t-test/ANOVA).
# - title: Descriptive title for reporting.
# - plot_type: Visualization type (box or heatmap).
# - Note: A/B pairs are selected as top 2 values by frequency or predefined; skips missing columns or insufficient unique values with warnings.

In [6]:
# Cell 5: Execute Hypothesis Tests with Equivalence Checks
# Initialize results dictionary
results = {}

# Add time tracking
import time
start_time = time.time()

# Loop through each hypothesis and perform tests
for i, (group_col, [group_a, group_b], metric_col, is_cat, title, plot_type) in enumerate(hypotheses):
    print(f"Processing hypothesis {i+1}/28: {title}")
    # Filter data to A/B groups
    df_temp = df[df[group_col[0]].isin([group_a, group_b])].copy()
    if df_temp.empty or df_temp[group_col[0]].nunique() < 2:
        print(f"Warning: Insufficient data for {title}, skipping.")
        continue
    
    # Equivalence checks on key attributes
    attributes_to_check = ["TotalPremium", "VehicleType"]  # Example attributes
    for attr in attributes_to_check:
        if attr in df_temp.columns:
            group_a_data = df_temp[df_temp[group_col[0]] == group_a][attr].dropna()
            group_b_data = df_temp[df_temp[group_col[0]] == group_b][attr].dropna()
            if len(group_a_data) > 0 and len(group_b_data) > 0:
                if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:
                    contingency = pd.crosstab(df_temp[group_col[0]], df_temp[attr])
                    if contingency.shape[1] > 1:  # Ensure valid contingency table
                        chi2, p_eq, dof, _ = stats.chi2_contingency(contingency)
                    else:
                        chi2, p_eq, dof, _ = (0, 1, 0, None)  # Fallback 4-tuple
                else:
                    t_stat, p_eq = stats.ttest_ind(group_a_data, group_b_data, equal_var=False) if len(group_a_data) > 1 and len(group_b_data) > 1 else (0, 1)
                if p_eq < 0.05:
                    print(f"Warning: {attr} differs significantly between {group_a} and {group_b} (p = {p_eq:.4f}), proceed with caution.")
    
    # Perform hypothesis test
    if plot_type == "heatmap":
        df_temp = df_temp[df_temp["TotalClaims"] > 0].copy()
    result, df_temp = test_hypothesis(df_temp, group_col[0], metric_col, is_cat)
    results[group_col[0] + "_" + str(metric_col) if not isinstance(metric_col, pd.Series) else group_col[0] + "_temp_metric"] = result
    if result:
        p_value = result.get("p_value", "N/A")
        print(f"{title} - P-value: {p_value}")

# End time tracking
print(f"Total execution time: {time.time() - start_time:.2f} seconds")

# Documentation
# - results: Stores test outcomes with A/B-specific results.
# - test_hypothesis: Applies statistical tests after filtering to A/B groups.
# - Equivalence Checks: Tests TotalPremium (numerical) and VehicleType (categorical) for similarity; warns if p < 0.05.
# - Note: Handles invalid contingency tables with a 4-tuple fallback; skips tests with insufficient data; adjust attributes_to_check based on dataset.

Processing hypothesis 1/28: Claim Severity by Province


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2035437510.py:25: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:


Claim Severity by Province - P-value: 0.01117179468533579
Processing hypothesis 2/28: Claim Severity by Zip Code


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2035437510.py:25: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:


Claim Severity by Zip Code - P-value: 0.393492808828808
Processing hypothesis 3/28: Claim Frequency by Gender
Claim Frequency by Gender - P-value: 1.0
Processing hypothesis 4/28: Margin by Zip Code


...

C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2035437510.py:25: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:


Claim Frequency by Premium Level - P-value: 1.0085945781954506e-136
Processing hypothesis 23/28: Claim Severity by Sum Insured


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2035437510.py:25: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:


Claim Severity by Sum Insured - P-value: 1.5856815917884146e-19
Processing hypothesis 24/28: Claim Severity by Month
Claim Severity by Month - P-value: 0.021006181796829802
Total execution time: 35.99 seconds


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\2035437510.py:25: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_temp[attr]) or df_temp[attr].nunique() <= 10:


In [8]:
# Cell 6: Generate Visualizations
import time
start_time = time.time()

# Loop through each hypothesis and generate visualizations
for i, (group_col, [group_a, group_b], metric_col, is_cat, title, plot_type) in enumerate(hypotheses):
    print(f"Generating visualization {i+1}/28: {title}")
    print(f"Original unique values in {group_col[0]} for {title}: {df[group_col[0]].nunique()}")
    # Filter data to A/B groups and check existence
    df_filtered = df[df[group_col[0]].isin([group_a, group_b])].copy()
    if df_filtered.empty or df_filtered[group_col[0]].nunique() < 2:
        print(f"Warning: Insufficient data for {group_a} and {group_b} in {title}, skipping.")
        continue
    if plot_type == "heatmap":
        df_filtered = df_filtered[df_filtered["TotalClaims"] > 0].copy()
    # Stratified sampling to ensure both A/B groups
    sample_size = min(5000, len(df_filtered))
    if sample_size > 0:
        df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(
            lambda x: x.sample(min(len(x), sample_size//2), random_state=42)
        ).reset_index(drop=True)
        print(f"Sampled unique values in {group_col[0]} for {title}: {df_temp[group_col[0]].nunique()}")
    else:
        print(f"Warning: No data to sample for {group_a} and {group_b} in {title}, skipping.")
        continue
    # Limit to top 50 groups if more than 50 unique values (though A/B limits this)
    if df_temp[group_col[0]].nunique() > 50:
        print(f"Reducing {group_col[0]} to top 50 groups")
        top_groups = df_temp[group_col[0]].value_counts().head(50).index
        df_temp = df_temp[df_temp[group_col[0]].isin(top_groups)]
    result = results.get(group_col[0] + "_" + str(metric_col) if not isinstance(metric_col, pd.Series) else group_col[0] + "_temp_metric", None)
    if result:
        # Modify test_hypothesis to skip Tukey for many groups
        def modified_test_hypothesis(df, group_col, metric_col, is_cat):
            temp_col = None
            if isinstance(metric_col, pd.Series):
                temp_col = "temp_metric"
                df = df.copy()
                df[temp_col] = metric_col
                metric_col = temp_col
            if is_cat:
                contingency = pd.crosstab(df[group_col], df['TotalClaims'] > 0)
                chi2, p, dof, expected = stats.chi2_contingency(contingency)
                return {'statistic': chi2, 'p_value': p, 'dof': dof}, df
            else:
                groups = [group[metric_col].dropna() for name, group in df.groupby(group_col, observed=True) if not group[metric_col].dropna().empty]
                if len(groups) == 2:
                    t_stat, p_value = stats.ttest_ind(groups[0], groups[1])
                    return {'statistic': t_stat, 'p_value': p_value}, df
                elif len(groups) > 2 and len(groups) <= 50:  # Only run ANOVA/Tukey if groups <= 50
                    anova = stats.f_oneway(*groups)
                    tukey = pairwise_tukeyhsd(df[metric_col].dropna(), df[group_col].dropna())
                    return {'anova_statistic': anova.statistic, 'p_value': anova.pvalue, 'tukey_results': tukey}, df
                else:
                    anova = stats.f_oneway(*groups) if groups else (0, 1)  # Fallback ANOVA without Tukey
                    return {'anova_statistic': anova[0], 'p_value': anova[1]}, df
        result, df_temp = modified_test_hypothesis(df_temp, group_col[0], metric_col, is_cat)
        visualize_results(df_temp, group_col[0], metric_col, title, plot_type if plot_type else "box")

print(f"Total visualization time: {time.time() - start_time:.2f} seconds")

# Documentation
# - visualize_results: Generates plots based on test outcomes.
# - Note: Uses stratified sampling (up to 5,000 rows) with observed=True to ensure A/B groups, limits groups to 50; skips Tukey HSD for >50 groups.
# - Adjustment: Filters to A/B groups and skips if data is insufficient; suppresses FutureWarnings.

Generating visualization 1/28: Claim Severity by Province
Original unique values in Province for Claim Severity by Province: 9


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\4046931547.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(


Sampled unique values in Province for Claim Severity by Province: 2
Generating visualization 2/28: Claim Severity by Zip Code
Original unique values in PostalCode for Claim Severity by Zip Code: 780


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\4046931547.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(


Sampled unique values in PostalCode for Claim Severity by Zip Code: 2
Generating visualization 3/28: Claim Frequency by Gender
Original unique values in Gender for Claim Frequency by Gender: 3


...

C:\Users\hp\AppData\Local\Temp\ipykernel_22132\4046931547.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(


Sampled unique values in TotalPremiumBin for Claim Frequency by Premium Level: 2
Generating visualization 23/28: Claim Severity by Sum Insured
Original unique values in SumInsuredBin for Claim Severity by Sum Insured: 4


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\4046931547.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(


Sampled unique values in SumInsuredBin for Claim Severity by Sum Insured: 2
Generating visualization 24/28: Claim Severity by Month
Original unique values in TransactionMonth for Claim Severity by Month: 23
Sampled unique values in TransactionMonth for Claim Severity by Month: 2
Total visualization time: 24.14 seconds


C:\Users\hp\AppData\Local\Temp\ipykernel_22132\4046931547.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_filtered.groupby(group_col[0], group_keys=False, observed=True).apply(


In [10]:
# Cell 7: Summarize Significant Results
# Filter and display significant hypotheses (p <= 0.05)
significant_results = {k: v for k, v in results.items() if v and v.get("p_value", 1) <= 0.05}
print("Significant Results (p <= 0.05):")
for hypo, result in significant_results.items():
    print("{0} - P-value: {1}".format(hypo, result["p_value"]))

# Documentation
# - significant_results: Highlights hypotheses with p <= 0.05.
# - print: Lists significant findings.

Significant Results (p <= 0.05):
Province_TotalClaims - P-value: 0.01117179468533579
MainCrestaZone_TotalClaims - P-value: 1.722470110839296e-06
SubCrestaZone_TotalClaims - P-value: 2.7242719626036405e-09
CoverCategory_TotalClaims - P-value: 0.0007963006407114917
TotalPremiumBin_TotalClaims - P-value: 1.0085945781954506e-136
SumInsuredBin_TotalClaims - P-value: 1.5856815917884146e-19
TransactionMonth_TotalClaims - P-value: 0.021006181796829802


In [12]:
# Cell 8: Detailed Interpretation and Recommendations
print("### Hypothesis Testing Results and Interpretations")
for key, result in results.items():
    if result and 'p_value' in result:
        decision = "Reject H₀ (Significant)" if result['p_value'] <= 0.05 else "Fail to Reject H₀ (Not Significant)"
        print(f"- {key}: P-value = {result['p_value']}, Decision: {decision}")
        if decision == "Reject H₀ (Significant)":
            if 'statistic' in result:
                print(f"  - Statistic: {result['statistic']}")
            # Interpretations and Recommendations
            if 'Province' in key:
                print("  - Interpretation: Significant difference in claim frequency between provinces (e.g., Gauteng vs. Western Cape), likely due to urban density or risk exposure.")
                print("  - Recommendation: Consider a 5-10% premium adjustment for high-claim provinces after A/B validation.")
            elif 'MainCrestaZone' in key or 'SubCrestaZone' in key:
                print("  - Interpretation: Zone-based differences in claims suggest geographic risk variations.")
                print("  - Recommendation: Adjust underwriting guidelines by zone, targeting 10-15% premium increases in high-risk areas.")
            elif 'CoverCategory' in key:
                print("  - Interpretation: Certain cover categories drive higher claims, possibly due to policy type or usage.")
                print("  - Recommendation: Review cover category pricing, potentially increasing by 5-10% for high-claim categories.")
            elif 'TotalPremiumBin' in key or 'SumInsuredBin' in key:
                print("  - Interpretation: Claim frequency varies significantly with premium or insured value bins, indicating risk tiering.")
                print("  - Recommendation: Implement tiered pricing with 10-20% increases for higher bins after further analysis.")
            elif 'TransactionMonth' in key:
                print("  - Interpretation: Seasonal claim patterns suggest time-based risk factors.")
                print("  - Recommendation: Adjust reserves or premiums by 5-10% for high-claim months (e.g., year-end).")
        else:
            print("  - Interpretation: No significant difference detected; current pricing or policies may be appropriate.")
            print("  - Recommendation: Maintain current strategy, but monitor for future trends.")
        print("  - Note: Validate with larger dataset or additional A/B tests.\n")

print("### General Recommendations")
print("- Revisit skipped hypotheses (e.g., PostalCode) with adjusted A/B pairs to achieve 28/28 coverage.")
print("- Conduct quarterly reviews to refine models based on new data.")
print("- Prioritize A/B validation for significant results before implementing premium changes.")

### Hypothesis Testing Results and Interpretations
- Province_TotalClaims: P-value = 0.01117179468533579, Decision: Reject H₀ (Significant)
  - Statistic: 2.5372883209254207
  - Interpretation: Significant difference in claim frequency between provinces (e.g., Gauteng vs. Western Cape), likely due to urban density or risk exposure.
  - Recommendation: Consider a 5-10% premium adjustment for high-claim provinces after A/B validation.
  - Note: Validate with larger dataset or additional A/B tests.

- PostalCode_TotalClaims: P-value = 0.393492808828808, Decision: Fail to Reject H₀ (Not Significant)
  - Interpretation: No significant difference detected; current pricing or policies may be appropriate.
  - Recommendation: Maintain current strategy, but monitor for future trends.
  - Note: Validate with larger dataset or additional A/B tests.

- Gender_TotalClaims: P-value = 1.0, Decision: Fail to Reject H₀ (Not Significant)
  - Interpretation: No significant difference detected; current pri

In [14]:
# Cell 9: Export Results

# Export results to CSV
results_df = pd.DataFrame([(k, v.get('p_value', 'N/A'), v.get('statistic', 'N/A'), 
                           "Reject H₀" if v.get('p_value', 1) <= 0.05 else "Fail to Reject H₀") 
                          for k, v in results.items()],
                         columns=['Hypothesis', 'P-value', 'Statistic', 'Decision'])
results_df.to_csv('task3_results.csv', index=False)
print("Results exported to task3_results.csv")

# Export analysis to MD with UTF-8 encoding
with open('task3_analysis.md', 'w', encoding='utf-8') as f:
    f.write("# Task 3 Analysis Report\n")
    f.write("## Hypothesis Testing Results\n")
    for key, result in results.items():
        if result and 'p_value' in result:
            decision = "Reject H₀" if result['p_value'] <= 0.05 else "Fail to Reject H₀"
            f.write(f"- {key}: P-value = {result['p_value']}, Decision: {decision}\n")
    f.write("## Interpretations and Recommendations\n")
    for key, result in results.items():
        if result and 'p_value' in result and result['p_value'] <= 0.05:
            f.write(f"- {key}\n")
            if 'Province' in key:
                f.write("  - Interpretation: Significant difference in claim frequency between provinces.\n")
                f.write("  - Recommendation: Consider a 5-10% premium adjustment for high-claim provinces.\n")
            # Add other conditions as in Cell 8
    f.write("## Notes\n- 4 hypotheses skipped (e.g., PostalCode); revisit A/B pairs for full 28/28 coverage.\n")
print("Analysis exported to task3_analysis.md")

Results exported to task3_results.csv
Analysis exported to task3_analysis.md
